# <font color='blue'>Data Science Academy - Visualização de Dados e Design de Dashboards</font>

## Mapas Interativos com Geoplotlib

In [1]:
!pip install ipykernel

In [2]:
### BAIXANDO ESSA VERSÃO, PORQUE A VERSÃO ATUAL 1.3 ESTÁ COM BUGs
!pip install pyglet==1.2.4 

    100% |████████████████████████████████| 972kB 8.8MB/s eta 0:00:01


In [3]:
!pip install geoplotlib

  Running setup.py bdist_wheel for geoplotlib ... done
  Stored in directory: /Users/dmpm/Library/Caches/pip/wheels/c1/66/6f/0004f2792164ceeacf6ec80766530cbc44555bf8e64c397d80
Successfully built geoplotlib


In [4]:
import geoplotlib

In [5]:
# Mapa de Densidade
from geoplotlib.utils import read_csv

data = read_csv('data/bus.csv')
geoplotlib.dot(data)
geoplotlib.show()

In [6]:
# Histograma 2D
from geoplotlib.utils import read_csv, BoundingBox

data = read_csv('data/opencellid_dk.csv')
geoplotlib.hist(data, colorscale = 'sqrt', binsize = 8)
geoplotlib.set_bbox(BoundingBox.DK)
geoplotlib.show()

In [7]:
# Pontos e Labels no Mapa
from geoplotlib.colors import colorbrewer
from geoplotlib.utils import epoch_to_str, BoundingBox, read_csv

data = read_csv('data/metro.csv')
geoplotlib.dot(data, 'r')
geoplotlib.labels(data, 'name', color = [0,0,255,255], font_size = 10, anchor_x = 'center')
geoplotlib.set_bbox(BoundingBox.KBH)
geoplotlib.show()

In [8]:
# Triangulação - delaunay
from geoplotlib.layers import DelaunayLayer
from geoplotlib.utils import read_csv, BoundingBox

data = read_csv('data/bus.csv')
geoplotlib.delaunay(data, cmap='hot_r')
geoplotlib.set_bbox(BoundingBox.DK)
geoplotlib.set_smoothing(True)
geoplotlib.show()

In [9]:
# Spatial Graph
from geoplotlib.utils import read_csv

data = read_csv('data/flights.csv')
geoplotlib.graph(data,
                 src_lat = 'lat_departure',
                 src_lon = 'lon_departure',
                 dest_lat = 'lat_arrival',
                 dest_lon = 'lon_arrival',
                 color = 'hot_r',
                 alpha = 16,
                 linewidth = 2)
geoplotlib.show()

In [10]:
# Choropleth
# Versão D3.js: https://bl.ocks.org/mbostock/4060606

from geoplotlib.utils import BoundingBox
from geoplotlib.colors import ColorMap
import json


# Taxa de desemprego através do país
def get_color(properties):
    key = str(int(properties['STATE'])) + properties['COUNTY']
    if key in unemployment:
        return cmap.to_color(unemployment.get(key), .15, 'lin')
    else:
        return [0, 0, 0, 0]


with open('data/unemployment.json') as fin:
    unemployment = json.load(fin)

cmap = ColorMap('Blues', alpha=255, levels=10)
geoplotlib.geojson('data/gz_2010_us_050_00_20m.json', fill = True, color = get_color, f_tooltip = lambda properties: properties['NAME'])
geoplotlib.geojson('data/gz_2010_us_050_00_20m.json', fill = False, color = [255, 255, 255, 64])
geoplotlib.set_bbox(BoundingBox.USA)
geoplotlib.show()

In [11]:
# Interação no Mapa através do teclado
import numpy as np
import pyglet
from geoplotlib.colors import create_set_cmap
from sklearn.cluster import KMeans
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.utils import BoundingBox

class KMeansLayer(BaseLayer):

    def __init__(self, data):
        self.data = data
        self.k = 2


    def invalidate(self, proj):
        self.painter = BatchPainter()
        x, y = proj.lonlat_to_screen(self.data['lon'], self.data['lat'])

        k_means = KMeans(n_clusters=self.k)
        k_means.fit(np.vstack([x,y]).T)
        labels = k_means.labels_

        self.cmap = create_set_cmap(set(labels), 'hsv')
        for l in set(labels):
            self.painter.set_color(self.cmap[l])
            self.painter.convexhull(x[labels == l], y[labels == l])
            self.painter.points(x[labels == l], y[labels == l], 2)
    
            
    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        ui_manager.info('Use esquerda e direita para aumentar / diminuir o número de clusters. k = %d' % self.k)
        self.painter.batch_draw()


    def on_key_release(self, key, modifiers):
        if key == pyglet.window.key.LEFT:
            self.k = max(2,self.k - 1)
            return True
        elif key == pyglet.window.key.RIGHT:
            self.k = self.k + 1
            return True
        return False
  

data = geoplotlib.utils.read_csv('data/bus.csv')
geoplotlib.add_layer(KMeansLayer(data))
geoplotlib.set_smoothing(True)
geoplotlib.set_bbox(geoplotlib.utils.BoundingBox.DK)
geoplotlib.show()

In [12]:
# Gráfico Animado com o percurso de um táxi
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.utils import epoch_to_str, BoundingBox, read_csv

class TrailsLayer(BaseLayer):

    def __init__(self):
        self.data = read_csv('data/taxi.csv')
        self.data = self.data.where(self.data['taxi_id'] == list(set(self.data['taxi_id']))[2])
        self.t = self.data['timestamp'].min()
        self.painter = BatchPainter()


    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        self.painter.set_color([0,0,255])
        df = self.data.where((self.data['timestamp'] > self.t) & (self.data['timestamp'] <= self.t + 30*60))
        proj.fit(BoundingBox.from_points(lons=df['lon'], lats=df['lat']), max_zoom=14)
        x, y = proj.lonlat_to_screen(df['lon'], df['lat'])
        self.painter.linestrip(x, y, 10)
        self.t += 30
        if self.t > self.data['timestamp'].max():
            self.t = self.data['timestamp'].min()

        self.painter.batch_draw()
        ui_manager.info(epoch_to_str(self.t))


geoplotlib.add_layer(TrailsLayer())
geoplotlib.show()

In [13]:
# Mapa com animação das linhas do metro
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
import geoplotlib
from geoplotlib.colors import colorbrewer
from geoplotlib.utils import epoch_to_str, BoundingBox, read_csv


class TrailsLayer(BaseLayer):

    def __init__(self):
        self.data = read_csv('data/taxi.csv')
        self.cmap = colorbrewer(self.data['taxi_id'], alpha=220)
        self.t = self.data['timestamp'].min()
        self.painter = BatchPainter()


    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        df = self.data.where((self.data['timestamp'] > self.t) & (self.data['timestamp'] <= self.t + 15*60))

        for taxi_id in set(df['taxi_id']):
            grp = df.where(df['taxi_id'] == taxi_id)
            self.painter.set_color(self.cmap[taxi_id])
            x, y = proj.lonlat_to_screen(grp['lon'], grp['lat'])
            self.painter.points(x, y, 10)

        self.t += 2*60

        if self.t > self.data['timestamp'].max():
            self.t = self.data['timestamp'].min()

        self.painter.batch_draw()
        ui_manager.info(epoch_to_str(self.t))


    def bbox(self):
        return BoundingBox(north=40.110222, west=115.924463, south=39.705711, east=116.803369)


geoplotlib.add_layer(TrailsLayer())
geoplotlib.show()

In [14]:
# Quadtree
import numpy as np
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.utils import BoundingBox


class QuadTree:

    def __init__(self, left, right, top, bottom):
        self.left = left
        self.right = right
        self.top = top
        self.bottom = bottom


    def split(self):
        middle_x = (self.left + self.right) / 2.
        middle_y = (self.top + self.bottom) / 2.
        return [QuadTree(self.left, middle_x, self.top, middle_y),
                QuadTree(middle_x, self.right, self.top, middle_y),
                QuadTree(self.left, middle_x, middle_y, self.bottom),
                QuadTree(middle_x, self.right, middle_y, self.bottom)]


    def can_split(self, x, y):
        if self.right - self.left < 4:
            return False

        mask = (x > self.left) & (x < self.right) & (y > self.bottom) & (y < self.top)
        return mask.any()


    def __repr__(self):
        return '(%.2f,%.2f,%.2f,%.2f)' % (self.left, self.right, self.top, self.bottom)


class QuadsLayer(BaseLayer):

    def __init__(self, data, cmap='hot_r'):
        self.data = data
        if cmap is not None:
            self.cmap = geoplotlib.colors.ColorMap(cmap, alpha=196)
        else:
            self.cmap = None
            

    def invalidate(self, proj):
        self.painter = BatchPainter()
        x, y = proj.lonlat_to_screen(self.data['lon'], self.data['lat'])
        
        w = x.max() - x.min()
        h = y.max() - y.min()
        w = np.ceil(w / 2) * 2
        h = np.ceil(h / 2) * 2
        l = max(w, h)
        
        root = QuadTree(x.min(), x.min() + l, y.min() + l, y.min())
        maxarea = (root.right - root.left) * (root.top - root.bottom)
        queue = [root]
        done = []
        while len(queue) > 0:
            qt = queue.pop()
            if qt.can_split(x, y):
                queue.extend(qt.split())
            else:
                done.append(qt)
        
        print (len(queue), len(done))

        if self.cmap is not None:
            for qt in done:
                area = (qt.right - qt.left) * (qt.top - qt.bottom)
                self.painter.set_color(self.cmap.to_color(1 + area, 1 + maxarea, 'log'))
                self.painter.rect(qt.left, qt.top, qt.right, qt.bottom)
        else:
            for qt in done:
                self.painter.linestrip([qt.left, qt.right, qt.right, qt.left],
                                       [qt.top, qt.top, qt.bottom, qt.bottom], closed=True)
    
            
    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter.batch_draw()
  

data = geoplotlib.utils.read_csv('data/bus.csv')
geoplotlib.add_layer(QuadsLayer(data, cmap=None))
geoplotlib.set_smoothing(False)
geoplotlib.set_bbox(geoplotlib.utils.BoundingBox.DK)
geoplotlib.show()

0 9904


## FIM